## LinkedIn API Access

In [111]:
from linkedin_v2 import linkedin

API_KEY   = ''
API_SECRET = ''

RETURN_URL = 'http://localhost:8888'

authentication = linkedin.LinkedInAuthentication(API_KEY, API_SECRET, RETURN_URL, ['r_liteprofile'])

print(authentication.authorization_url)  # open this url on your browser

https://www.linkedin.com/uas/oauth2/authorization?response_type=code&client_id=77y00e34lm3qsk&scope=r_liteprofile&state=d0aa0566616d0f0a7e1181d5afbc727b&redirect_uri=http%3A//localhost%3A8888


## Retrieving your LinkedIn profile

In [114]:
authentication.authorization_code = ''
result = authentication.get_access_token()

In [117]:
app = linkedin.LinkedInApplication(token=result.access_token)

In [118]:
app

## Download your profile data and read in connections data as a CSV file

In [43]:
import os
import csv

# Point this to your 'Connections.csv' file.
CSV_FILE = os.path.join('data', 'Connections.csv')

csvReader = csv.DictReader(open(CSV_FILE, encoding='utf-8-sig'), delimiter=',', quotechar='"')
contacts = [row for row in csvReader]

In [44]:
contacts

[OrderedDict([('First Name', 'Kim'),
              ('Last Name', 'Gammeltoft'),
              ('Email Address', ''),
              ('Company', 'NNE'),
              ('Position', 'Senior Project Manager & Team Leader'),
              ('Connected On', '03.12.2018')]),
 OrderedDict([('First Name', 'Kolja'),
              ('Last Name', 'Klein'),
              ('Email Address', ''),
              ('Company', 'NNE'),
              ('Position', 'Lead Engineer'),
              ('Connected On', '27.11.2018')]),
 OrderedDict([('First Name', 'Klaudia'),
              ('Last Name', 'Knozowska'),
              ('Email Address', ''),
              ('Company', ''),
              ('Position', ''),
              ('Connected On', '07.11.2018')]),
 OrderedDict([('First Name', 'Yavuz'),
              ('Last Name', 'Baykara'),
              ('Email Address', ''),
              ('Company', 'NNE'),
              ('Position', 'Director Project Governance'),
              ('Connected On', '02.11.2018')]),
 Ord

## Simple normalization of company suffixes from address book data

In [45]:
from prettytable import PrettyTable # pip install prettytable
from collections import Counter
from operator import itemgetter

# Define a set of transforms that converts the first item
# to the second item. Here, we're simply handling some
# commonly known abbreviations, stripping off common suffixes, 
# etc.

transforms = [(', Inc.', ''), (', Inc', ''), (', LLC', ''), (', LLP', ''),
               (' LLC', ''), (' Inc.', ''), (' Inc', '')]

companies = [c['Company'].strip() for c in contacts if c['Company'].strip() != '']

for i, _ in enumerate(companies):
    for transform in transforms:
        companies[i] = companies[i].replace(*transform)

pt = PrettyTable(field_names=['Company', 'Freq'])
pt.align = 'l'
c = Counter(companies)

[pt.add_row([company, freq]) for (company, freq) in sorted(c.items(), key=itemgetter(1), reverse=True) if freq > 1]

print(pt)

+-----------------------------------------+------+
| Company                                 | Freq |
+-----------------------------------------+------+
| NNE                                     | 22   |
| National Technical University of Athens | 3    |
| NNE AG                                  | 2    |
| Zaha Hadid Architects                   | 2    |
| EM2N                                    | 2    |
| Hodkinson Consultancy Ltd               | 2    |
+-----------------------------------------+------+


## Standardizing common job titles and computing their frequencies

In [46]:
transforms = [
    ('Sr.', 'Senior'),
    ('Sr', 'Senior'),
    ('Jr.', 'Junior'),
    ('Jr', 'Junior'),
    ('CEO', 'Chief Executive Officer'),
    ('COO', 'Chief Operating Officer'),
    ('CTO', 'Chief Technology Officer'),
    ('CFO', 'Chief Finance Officer'),
    ('VP', 'Vice President'),
    ]

# Read in a list of titles and split apart
# any combined titles like "President/CEO."
# Other variations could be handled as well, such
# as "President & CEO", "President and CEO", etc.

titles = []
for contact in contacts:
    titles.extend([t.strip() for t in contact['Position'].split('/')
                  if contact['Position'].strip() != ''])

# Replace common/known abbreviations

for i, _ in enumerate(titles):
    for transform in transforms:
        titles[i] = titles[i].replace(*transform)

# Print out a table of titles sorted by frequency

pt = PrettyTable(field_names=['Job Title', 'Freq'])
pt.align = 'l'
c = Counter(titles)
[pt.add_row([title, freq]) 
 for (title, freq) in sorted(c.items(), key=itemgetter(1), reverse=True) 
     if freq > 1]
print(pt)

# Print out a table of tokens sorted by frequency

tokens = []
for title in titles:
    tokens.extend([t.strip(',') for t in title.split()])
pt = PrettyTable(field_names=['Token', 'Freq'])
pt.align = 'l'
c = Counter(tokens)
[pt.add_row([token, freq]) 
 for (token, freq) in sorted(c.items(), key=itemgetter(1), reverse=True) 
     if freq > 1 and len(token) > 2]
print(pt)

+---------------------+------+
| Job Title           | Freq |
+---------------------+------+
| Architect           | 20   |
| Junior Architect    | 4    |
| Project Architect   | 4    |
| Project Engineer    | 3    |
| Architekt           | 3    |
| Architect Engineer  | 3    |
| Senior Architect    | 3    |
| Assistant Professor | 3    |
| Lead Engineer       | 2    |
| Director            | 2    |
| Teaching Assistant  | 2    |
| Designer            | 2    |
| Urban Designer      | 2    |
| Founder             | 2    |
| Co-Founder          | 2    |
+---------------------+------+
+----------------+------+
| Token          | Freq |
+----------------+------+
| Architect      | 43   |
| Engineer       | 21   |
| Project        | 17   |
| Senior         | 13   |
| Assistant      | 12   |
| Manager        | 10   |
| Designer       | 8    |
| Director       | 7    |
| Specialist     | 7    |
| Professor      | 7    |
| Design         | 7    |
| Junior         | 5    |
| Research       | 5 